# Small example to show the influence of the prevalence on the results of a COVID-19 test
This example is inspired by: https://www.quarks.de/gesundheit/medizin/corona-test-wie-funktioniert-der-test/ and https://www.tagesschau.de/faktenfinder/corona-test-117.html

In [30]:
# Housekeeping: Loading of necessary packets
import numpy as np
from scipy import stats
import plotly as py
import plotly.graph_objs as go
import plotly.express as px

## The numbers for our experiment
We use the estimated numbers of active COVID-19 cases in Germany from the RKI website.
As test we assume a test by Roche.

In [15]:
# As of Spe. 16.09 there are approx. 20.000 active COVID-19 cases in Germany
# https://experience.arcgis.com/experience/478220a4c454480e823b17327b2bf1d4/page/page_0/
cum_cases = 263663
recovered_cases = 236000
deaths = 9368
inhabitants = 83149300 # https://en.wikipedia.org/wiki/Demographics_of_Germany

# Assuming a test with perfect sensitivity
sensitivity = 1

In [26]:
current_cases = cum_cases - recovered_cases - deaths
prevalence = current_cases/inhabitants
print('In Germany we have {} current cases and a prevalence of {:.2f} per 100,000 inhabitants'.
      format(current_cases, prevalence*100000))

In Germany we have 18295 current cases and a prevalence of 22.00 per 100,000 inhabitants


## Calculating the number of people we need to test to have a 50% probability of finding an active case
If we randomly test people, how many do we need to test to have a 50% probability of finding an active case.

Using the binomial distribution

$P(p|k,n)=\binom{n}{k} p^k (1-p)^{n-k}$.

The probability of not finding any case in $n$ tests is
$P(k=0 \mid p, n) = \binom{n}{0} p^0 (1-p)^{n} = (1-p)^{n}$.

This is the inverse probability of finding at least one case. Hence

$P(k \geq 1 \mid p, n) = (1-p)^{n} \geq 0.5$ .

Hence

$n \geq \frac{\ln 0.5}{\ln(1-p)}$

where $p$ is the prevalence of the cases.


In [39]:
required_n = np.log(0.5)/np.log(1-prevalence)
print('If we test randomly, we need to test {:d} people to have a 50% probability of detecting an active case'.format(np.int(required_n)))

If we test randomly, we need to test 3149 people to have a 50% probability of detecting an active case


## Calculating the required specificity of a COVID-19 test

We assume a test with prefect sensitivity (1.0) and want to know what the necessary specificity is to have a 50% probability for a random person to actually be an active COVID-19 case after being tested positive

Lets use the term $\text{P}^+$ to indicate that a person is infected and $\text{P}^-$ that he or she is not. And with $\text{test}^+$ we indicate that the test was positive and with $\text{test}^-$ that it was negative.

We start with Bayes theorem and then apply it to our case. 

$P(A\mid B) =\frac{P(B\mid A)P(A)}{\sum_A P(B\mid A)}$

The sensitivity is the probability that the test was positive if the person was positive ($P(\text{test}^+\mid \text{P}^+)$). 

The specificity is the probability that the test was negative if the person was negative ($P(\text{test}^-\mid \text{P}^-)$). 

Hence the probability that the test was positive if the person was negative is $1-\text{specificity}$ ($P(\text{test}^+\mid \text{P}^-) = 1 - P(\text{test}^-\mid \text{P}^-)$)


$P(\text{P}^+ \mid \text{test}^+) =\frac{P(\text{test}^+\mid \text{P}^+)P(\text{P}^+)}{P(\text{test}^+\mid \text{P}^+)P(\text{P}^+)+P(\text{test}^+ \mid \text{P}^-)P(\text{P}^-)}$ 

$P(\text{P}^+ \mid \text{test}^+) =\frac{\text{sensitivity} \cdot \text{prevalence}}{\text{sensitivity} \cdot \text{prevalence}+(1-\text{specificity}) \cdot (1-\text{prevalence})}$

So to find the correct specificity we need $P(\text{P}^+ \mid \text{test}^+) \geq 0.5$.

$\frac{\text{sensitivity} \cdot \text{prevalence}}{\text{sensitivity} \cdot \text{prevalence}+(1-\text{specificity}) \cdot (1-\text{prevalence})} \geq 0.5$

$\text{sensitivity} \cdot \text{prevalence} \geq 0.5(\text{sensitivity} \cdot \text{prevalence}+(1-\text{specificity}) \cdot (1-\text{prevalence}))$

$0.5(\text{sensitivity} \cdot \text{prevalence}) \geq   0.5(1-\text{specificity}) \cdot (1-\text{prevalence})$

$\frac{\text{sensitivity} \cdot \text{prevalence}}{1-\text{prevalence}} \geq (1-\text{specificity})$

$\text{specificity} \geq 1 - \frac{\text{sensitivity} \cdot \text{prevalence}}{1-\text{prevalence}}$

More information on COVID-19 tests is for example available in Sethuraman, N., Jeremiah, S. S., & Ryo, A. (2020). Interpreting Diagnostic Tests for SARS-CoV-2. In JAMA - Journal of the American Medical Association (Vol. 323, Issue 22, pp. 2249–2251). 

In [19]:
min_specificity = 1 - (sensitivity*prevalence)/(1-prevalence)
print('Required specificity is {} (given a prevalence of {} and sensitivity of {}'.format(min_specificity, prevalence, sensitivity))

Required specificity is 0.9997799256727379 (given a prevalence of 0.0002200259052090637 and sensitivity of 1


## Calculating the probability of having COVID-19 antibodies after an antibody test was positive
For this case the prevalence is quite different as it is based on all people which were infected (and survived)

In [42]:
# Adapting the prevalence
relevant_cases = cum_cases -deaths
prevalence = relevant_cases/inhabitants

# Test characteristics of Roche test
# https://www.roche.de/diagnostics/tests-parameter/elecsys-anti-sars-cov-2.html#Allgemeine-Informationen-zu-SARS-CoV-2
specificity = 0.998
sensitivity = 0.995

print('In Germany we have {} cases which survived COID-19, resulting in a prevalence of {:.2f} per 100,000 inhabitants'.
      format(relevant_cases, prevalence*100000))

P_COVID19_antibodies = (sensitivity*prevalence)/(sensitivity*prevalence+(1-specificity)*(1-prevalence))
print('The probability for a random person of having COVID-19 antibodies after a positive test is {}, i.e. {:.0f}%'.format(P_COVID19_antibodies, P_COVID19_antibodies*100))

In Germany we have 254295 cases which survived COID-19, resulting in a prevalence of 305.83 per 100,000 inhabitants
The probability for a random person of having COVID-19 antibodies after a positive test is 0.6041436196659687, i.e. 60%


## Now lets assume we use the test from Bencard
The Bencard test has different parameters for sensitivity and specificity

In [43]:
# Test characteristics
# https://bencard.ch/portfolio/covid-19-igg-und-igm-elisa-kits/
specificity = 0.998
sensitivity = 0.984

In [44]:
P_having_COVID19 = (sensitivity*prevalence)/(sensitivity*prevalence+(1-specificity)*(1-prevalence))
print(P_having_COVID19)

0.601481927546635
